# Linear Regression

In this tutorial we will implement a linear regression model. We will also implement a function that splits the available data into a training and a testing part.

## Problem Setting

We will use the Boston Housing Dataset. This dataset contains information collected by the U.S Census Service concerning housing in the city of Boston in the state of Massachusetts in 1978. Our goal is to predict the median value of the houses in a particular town in the city of Boston given its attributes. Check the file ’housing_features_description.txt’ for more information on the attributes.

In [1]:
import urllib
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

### NOTE: The boston dataset has an ethical problem. More on this can e.g. be found in the scikit documentation. ###
### Summary: The dataset contains the proportion of black people, which was assumed that racial self-segregation had a positive impact on house prices. ###


data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
boston_data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
boston_target = raw_df.values[1::2, 2]

df=pd.DataFrame(boston_data)
df.columns=['crime_rate','res_land_zoned','industry','charles_river','nox','avg_num_rooms','prop_bf_1940','dst_emply_center','rd_highway_idx','tax_rate','stdnt_tchr_ratio','prop_blacks','low_status_pct']
X=boston_data
y=boston_target


In [2]:
df.head(10)

,crime_rate,res_land_zoned,industry,charles_river,nox,avg_num_rooms,prop_bf_1940,dst_emply_center,rd_highway_idx,tax_rate,stdnt_tchr_ratio,prop_blacks,low_status_pct
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10


# Note 1:

Think about the ethical aspects of this dataset and machine learning in general. 

Can you always trust your data source? Can we use every possible information for our models?

### Exercise 1

Write the *split_train_test(X,y,split,seed)*, given an instance matrix $X \in \mathbb{R}^{N \times D}$, labels $y \in Y^N$, a split ratio in $[0, 1]$ and a random seed $\in \mathbb{Z}$. Split the dataset in $(split×100)\%$ of the instances for training our model and the rest for testing, i.e. 

$$ \left|X_{\text{train}}\right| = \lceil \text{split} \cdot N \rceil, \qquad |X_{\text{train}}| + |X_{\text{test}}| = N. $$
Make sure you use the given random number generator seed so we all get the same results. The function is supposed to return:

- X_train, y_train: the training instances and labels;
- X_test, y_test: the test instances and labels,

in the same order as was mentioned.

Hint: It may be helpful to use shuffling functionality (e.g. np.random.shuffle).

In [6]:
def split_train_test(X,y,split,seed):
    np.random.seed(seed)
    
    # Shuffle the indices of the instances
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    print(indices)
    
    # Determine the number of instances for training and testing
    num_train = int(np.ceil(split * X.shape[0]))
    num_test = X.shape[0] - num_train
    
    # Split the instances and labels based on the shuffled indices
    X_train = X[indices[:num_train]]
    y_train = y[indices[:num_train]]
    X_test = X[indices[num_train:]]
    y_test = y[indices[num_train:]]
    
    return X_train, y_train, X_test, y_test

In [7]:
# Create an example dataset
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])
y = np.array([0, 1, 0, 1, 1])

# Split the dataset using the function
split = 0.6  # 60% for training, 40% for testing
seed = 123  # Random seed for reproducibility

X_train, y_train, X_test, y_test = split_train_test(X, y, split, seed)

# Print the results
print("X_train:\n", X_train)
print("y_train:", y_train)
print("X_test:\n", X_test)
print("y_test:", y_test)


[1 3 4 0 2]
X_train:
 [[ 3  4]
 [ 7  8]
 [ 9 10]]
y_train: [1 1 1]
X_test:
 [[1 2]
 [5 6]]
y_test: [0 0]


### Exercise 2

Write the function *train_linear_reg(X_train,y_train,lmbd)*.
Implement the ridge regression model (slide 24). The function should output the learned weight vector $\theta \in \mathbb{R}^D$ or $\mathbb{R}^{D+1}$ (depending on whether you are adding *bias*).

In [8]:
def train_linear_reg(X, y, lmbd):
    # Add a column of ones to the feature matrix for the bias term
    X_train_extended = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    
    # Compute the weight vector using ridge regression formula
    A = np.dot(X_train_extended.T, X_train_extended) + lmbd * np.identity(X_train_extended.shape[1])
    b = np.dot(X_train_extended.T, y_train)
    theta = np.linalg.solve(A, b)
    
    return theta

### Exercise 3

Write the function *predict(X,theta)* which predicts housing values vector pred for a dataset X and a previously trained parameter vector $\theta$.

In [14]:
def predict(X, theta):
    X_extended = np.c_[np.ones((X.shape[0], 1)), X]
    
    # Predict the values using the learned parameter vector
    y_pred = np.dot(X_extended, theta)
    
    return y_pred 

### Exercise 4

Write the function *mean_abs_loss(y_true,y_pred)* which computes the mean of the absolute differences between our prediction vector $y\_pred$ and the real housing values $y\_true$.

In [12]:
def mean_abs_loss(y_true,y_pred):
    
    # Compute the absolute differences between the predicted and true values
    abs_diff = np.abs(y_pred - y_true)
    
    # Compute the mean of the absolute differences
    mean_loss = np.mean(abs_diff)
    
    return mean_loss

### Exercise 5

Evaluate your solutions by running the following code. 

Moreover, answer the following questions: What is the most important feature in your model? Are there features that are not so important? What happens if you remove them? Are there outliers with a high absolute loss?

In [15]:
seed = 3
lmbd=1
split=0.7
X_train,y_train,X_test,y_test=split_train_test(X,y,split,seed)
theta=train_linear_reg(X_train,y_train,lmbd)
y_pred=predict(X_test,theta)
mae=mean_abs_loss(y_test,y_pred)
print ('The mean absolute loss is {loss:0.3f}'.format(loss=mae*1000))

[3 4 1 0 2]
The mean absolute loss is 729.295


In [20]:
import numpy as np

# Example data
X = np.random.rand(5, 3)  # Randomly generated features
y = np.random.rand(5)  # Randomly generated target variable
print(X,y)
# Add a column of ones to the feature matrix for the bias term
X_extended = np.c_[np.ones((X.shape[0], 1)), X]
print(X_extended)
# Calculate theta using the least squares method
theta = np.linalg.inv(X_extended.T @ X_extended) @ X_extended.T @ y

print("Parameter vector theta:")
print(theta)


[[0.46921516 0.89149934 0.10334615]
 [0.32389346 0.89590584 0.82739969]
 [0.53125337 0.31099265 0.13791257]
 [0.84410415 0.23672691 0.71072713]
 [0.40703813 0.60822164 0.16176878]] [0.57206365 0.84205745 0.41975487 0.53378787 0.21738465]
[[1.         0.46921516 0.89149934 0.10334615]
 [1.         0.32389346 0.89590584 0.82739969]
 [1.         0.53125337 0.31099265 0.13791257]
 [1.         0.84410415 0.23672691 0.71072713]
 [1.         0.40703813 0.60822164 0.16176878]]
Parameter vector theta:
[-0.03722684  0.23056056  0.45591626  0.43039005]
